In [ ]:
import uproot
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats

In [ ]:
savedir = "/icarus/app/users/gputnam/calib/plots/"
dosave=True

In [ ]:
branches = [
    "meta.run",
    "meta.evt",
    "meta.iproc",
    "t0",
    "id",
    "clear_cosmic_muon",
    "start_x",
    "start_y",
    "start_z",
    "end_x",
    "end_y",
    "end_z",
    "dir_x",
    "dir_y",
    "dir_z",
    "hit_min_time_p2_tpcE",
    "hit_max_time_p2_tpcE",
    "hit_min_time_p2_tpcW",
    "hit_max_time_p2_tpcW"
]

In [ ]:
# fname = "icarus.root"
fname = "/icarus/app/users/gputnam/calib/proc/caloanaout_run5458_N1000_procF_all.root"
folderE = "caloanaCryoE"
folderW = "caloanaCryoW"
tname = "GCalorimetryAnalyzer"

treeE = uproot.open(fname)[folderE][tname]
data = treeE.arrays(branches, library="pd")

In [ ]:
data

In [ ]:
#vdrift = 0.1574 # cm / us
#driftV = 0.156267 # cm / us


a2c_dist = 148.275 # cm
tanode = 850 # ticks
tick_period = 400 # ns
NTICKS = 4096 # ticks

In [ ]:
deltat_E = data.hit_max_time_p2_tpcE - data.hit_min_time_p2_tpcE
deltat_W = data.hit_max_time_p2_tpcW - data.hit_min_time_p2_tpcW

In [ ]:
# define the fit function
def fit_readout_window(tick, *p):
    N = p[0]
    sigma = p[1]
    center = p[2]
    beta = p[3]
    m = p[4]
    return N*stats.crystalball.pdf(tick, beta, m, center, sigma)

In [ ]:
bins = np.linspace(2200, 2500, 61)
#bins=None
NEE,_,_ = plt.hist(deltat_E, bins=bins, histtype="step", label="TPC EE Data", linewidth=2)

bin_center = (bins[1:] + bins[:-1]) / 2.
points = np.linspace(bins[0], bins[-1], 101)

#_ = plt.axline((deltat, 0), (deltat, np.max(N)), color="r")

p0 = [3000, 10, 2300., 1, 2.]
popt, _ = curve_fit(fit_readout_window, bin_center, NEE, p0=p0)
plt.plot(points, fit_readout_window(points, *popt), label="TPC EE Fit")
drift_length_EE = popt[2]

NEW,_,_ = plt.hist(deltat_W, bins=bins, histtype="step", label="TPC EW", linewidth=2)

p0 = [5e5, 10, 2300., 1, 2.]
popt, _ = curve_fit(fit_readout_window, bin_center, NEW, p0=p0, maxfev=int(1e5))
plt.plot(points, fit_readout_window(points, *popt), label="TPC EW Fit")
drift_length_EW = popt[2]

# drift_length_EE = bin_center[np.argmax(NEE)]
# drift_length_EW = bin_center[np.argmax(NEW)]
drift_vel_EE = a2c_dist / (drift_length_EE * tick_period/1e3)
drift_vel_EW = a2c_dist / (drift_length_EW * tick_period/1e3)

handles, labels = plt.gca().get_legend_handles_labels()
order = [2,0,3,1]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])

plt.text(0.65, 0.2, "TPC EE Drift V:\n%.4f cm/$\\mu$s\n\nTPC WW Drift V:\n%.4f cm/$\\mu$s" % (drift_vel_EE, drift_vel_EW), transform=plt.gca().transAxes, fontsize=14)

plt.ylabel("Tracks")
plt.xlabel("Track Hit Time Difference [ticks]")

if dosave: plt.savefig(savedir + "track_drift_dist.png")

In [ ]:
deltat_cut = 2340
tcathode_EE = tanode + drift_length_EE
tcathode_EW = tanode + drift_length_EW
print(tcathode_EE, tcathode_EW)

In [ ]:
var = data.hit_max_time_p2_tpcE - data.hit_max_time_p2_tpcW

bins = np.linspace(-100, 100, 41)

N,_,_ = plt.hist(var, bins=bins, label="Data")
plt.xlabel("EE TPC Max Time - EW TPC Max Time [ticks]")
plt.ylabel("Cathode Crossing Tracks")
bin_centers = (bins[1:] + bins[:-1]) / 2.

def gauss(x, *p):
    A, mu, sigma = p
    return A * np.exp(-(x-mu)**2/(2*sigma**2))

p0 = [1., 0., 1.]

when_fit = (bin_centers > -25) & (bin_centers < 25)

coeff, var_matrix = curve_fit(gauss, bin_centers[when_fit], N[when_fit], p0=p0)
print(coeff)

plt.plot(bin_centers, gauss(bin_centers, *coeff), label="Gaussian Fit")
plt.legend()

tpc_toff = coeff[1]

plt.text(0.65, 0.6, "Offset:\n%.2f ticks" % tpc_toff, transform=plt.gca().transAxes, fontsize=14)

if dosave: plt.savefig(savedir + "cathode_crossing_tracks_toff.png")

In [ ]:
tpcE_is_best = deltat_E > deltat_W
toff_besttpc = data.hit_min_time_p2_tpcW - tanode
toff_besttpc[tpcE_is_best] =  data.hit_min_time_p2_tpcE - tanode

deltat_besttpc = np.maximum(deltat_E, deltat_W)

In [ ]:
data["c2a_t0"] = toff_besttpc * tick_period

In [ ]:
ccross_t0_E = data.hit_max_time_p2_tpcE - tcathode_EE
ccross_t0_W = data.hit_max_time_p2_tpcW - tcathode_EW

data["ccross_t0"] = ((ccross_t0_E + ccross_t0_W) / 2.) * tick_period

In [ ]:
has_t0 = True

In [ ]:
c2a = (deltat_besttpc > deltat_cut) #& has_t0 & two_tpc_0

In [ ]:
var = ((data["ccross_t0"] - data["c2a_t0"])/1e3)

when = c2a
bins = None#np.linspace(5, 45, 21)
bins = np.linspace(-20, 20, 21)
_ = plt.hist(var[when & ~tpcE_is_best], bins=bins, label="TPC EW", histtype="step")
_ = plt.hist(var[when &  tpcE_is_best], bins=bins, label="TPC EE", histtype="step")
plt.legend()
plt.xlabel("Cathode Cross T0 - Anode 2 Cathode T0 [us]")
plt.ylabel("Tracks")

if dosave: plt.savefig(savedir + "t0_comparison.png")

In [ ]:
var = ((data["t0"] - data["c2a_t0"])/1e3)

when = c2a
bins = np.linspace(0, 40, 21)
_ = plt.hist(var[when & ~tpcE_is_best], bins=bins, label="TPC EW", histtype="step")
_ = plt.hist(var[when &  tpcE_is_best], bins=bins, label="TPC EE", histtype="step")
plt.legend()
plt.xlabel("Cathode Cross (Pandora) T0 - Anode 2 Cathode T0 [us]")
plt.ylabel("Tracks")
if dosave: plt.savefig(savedir + "t0_comparison_pandora.png")

In [ ]:
var = ((data["ccross_t0"] - data.t0)/1e3)

when = has_t0
bins = np.linspace(-80, 0, 21)
_ = plt.hist(var[when & ~tpcE_is_best], bins=bins, label="TPC EW", histtype="step")
_ = plt.hist(var[when &  tpcE_is_best], bins=bins, label="TPC EE", histtype="step")
plt.legend(loc="upper left")
plt.xlabel("Cathode Cross T0: Mine - Pandora [us]")
plt.ylabel("Tracks")
if dosave: plt.savefig(savedir + "t0_comparison_pandora_cathode_cross.png")